In [1]:
import sapien.core as sapien
import numpy as np

# from jax.config import config
# config.update("jax_debug_nans", True)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130


In [2]:
sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

stabled = False
def create_scene(timestep, visual):
    s = sim.create_scene([0,0,0])
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = True
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../../../assets/Arm/panda.urdf")

    return s, robot

sim_timestep = 1/60
s0, robot = create_scene(sim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)


In [3]:
if not stabled:
    for _ in range(3000):
        s0.step()
    stabled = True

In [4]:
ctrl = np.load('ctrl.npy')

In [5]:
render_controller.show_window()
for u in ctrl:
    robot.set_qf(u)
    s0.step()
    s0.update_render()
    render_controller.render()
